In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import pandas as pd
import numpy as np
from datetime import datetime
import time
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=2)


from darts import TimeSeries
from darts.models import (NaiveSeasonal, NaiveDrift, Prophet,
                          ExponentialSmoothing, ARIMA, AutoARIMA,
                          RegressionModel, Theta, FFT)
from darts.utils.utils import ModelMode, SeasonalityMode, TrendMode
from darts.metrics import mape, mase, r2_score, smape
from darts.utils.statistics import check_seasonality, plot_acf, plot_residuals_analysis, extract_trend_and_seasonality

import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)

In [ ]:
df = pd.read_csv("../../data/later/profile_growth.csv")
df['Date'] = pd.to_datetime(df['Date'])



In [ ]:
# Create a TimeSeries, specifying the time and value columns
series = TimeSeries.from_dataframe(df, 'Date', 'Followers')

In [ ]:
series.plot()

## Creating a training and validation series
First, let's split our `TimeSeries` into a training and a validation series. Note: in general, it is also a good practice to keep a test series aside and never touch it until the end of the process. Here, we just build a training and a test series for simplicity.

#### Validation set is for 2 weeks

In [ ]:
train, val = series.split_after(df.shape[0] - 14)#pd.Timestamp('20220214'))
train.plot(label='training')
val.plot(label='validation')
plt.legend()

## Quickly try a few more models
Let's train a few more and compute their respective MAPE on the validation set:

In [ ]:
def eval_model(model):
    model.fit(train)
    forecast = model.predict(len(val))
    print('model {} obtains MAPE: {:.2f}%'.format(model, mape(val, forecast)))


eval_model(ExponentialSmoothing())
eval_model(Prophet())
eval_model(AutoARIMA())
eval_model(Theta())
eval_model(FFT())

### Scaled

In [ ]:
from darts.dataprocessing.transformers import Scaler

def eval_model(model):
    scaler_dfp = Scaler()
    series_dfp_scaled = scaler_dfp.fit_transform(series)
    train, val = series_dfp_scaled[:-8], series_dfp_scaled[-8:]
    model.fit(train)
    forecast = model.predict(len(val))
    print('model {} obtains MAPE: {:.2f}%'.format(model, mape(val, forecast)))

eval_model(ExponentialSmoothing())
eval_model(Prophet())
eval_model(AutoARIMA())
#eval_model(Theta())
eval_model(FFT())

In [ ]:
from darts.dataprocessing.transformers import BoxCox
from darts.utils.utils import ModelMode, SeasonalityMode, TrendMode


def eval_model(model):
    BoxCox_dfp = BoxCox()
    series_dfp_BoxCox = BoxCox_dfp.fit_transform(series)
    train_bx, val_bx = series_dfp_BoxCox.split_after(pd.Timestamp('20220214'))
    model.fit(train_bx)
    forecast = model.predict(len(val_bx))
    print('model {} obtains MAPE: {:.2f}%'.format(model,
                                                  mape(val_bx, forecast)))


eval_model(ExponentialSmoothing())
eval_model(Prophet())
eval_model(AutoARIMA())
eval_model(Theta(season_mode=SeasonalityMode.ADDITIVE))
eval_model(FFT())

Here, we did only built these models with their default parameters. We can probably do better if we fine-tune to our problem. Let's try with the Theta method.

### Gridsearch: - FFT

In [ ]:
BoxCox_dfp = BoxCox()
series_dfp_BoxCox = BoxCox_dfp.fit_transform(series)

fft_model = FFT()
parameters = {
    'nr_freqs_to_keep': np.arange(start=1, stop=20),
    'trend': ['poly', 'exp'],
    'trend_poly_degree': np.arange(start=1, stop=20)
}
fft_model.gridsearch(parameters,
                     series_dfp_BoxCox,
                     last_points_only=True,
                     metric=smape,
                     forecast_horizon=2)

In [ ]:
fft_model = FFT(nr_freqs_to_keep=7, trend='poly', trend_poly_degree=4)

In [ ]:
BoxCox_dfp = BoxCox()
series_dfp_BoxCox = BoxCox_dfp.fit_transform(series)
train_bx, val_bx = series_dfp_BoxCox.split_after(df.shape[0] - 14)#split_after(pd.Timestamp('20210131'))
fft_model.fit(train_bx)
preds = fft_model.predict(len(val_bx))
print('model {} obtains MAPE: {:.2f}%'.format(fft_model, mape(val_bx, preds)))

### Forecast

In [ ]:
forecast = BoxCox_dfp.inverse_transform(preds)

In [ ]:
train.plot(label='train')
val.plot(label='true')
forecast.plot(label='prediction')
plt.legend()

We can observe that the model with `best_theta` is so far the best we have, in terms of MAPE.

## Backtesting: simulate historical forecasting
So at this point we have a model that performs well on our validation set, and that's good. But how can we know the performance we *would have obtained* if we *had been using this model* historically. 

Backtesting simulates predictions that would have been obtained historically with a given model. It can take a while to produce, since the model is re-fit every time the simulated prediction time advances.

Such simulated forecasts are always defined with respect to a *forecast horizon*, which is the number of time steps that separate the prediction time from the forecast time. In the example below, we simulate forecasts done for 3 months in the future (compared to prediction time).

Using the `backtest()` method, you can either look at the performance of the model evaluated over the whole forecasts it produces (each point in each forecast is used to compute an error score) or only the last point of each historical forecast.
In the latter case, you can get a time series representation of those points by calling `historical_forecasts()` with the default setting (`last_points_only=True`)

In [ ]:
#! pip install ipywidgets
#!jupyter nbextension enable --py widgetsnbextension
#from ipywidgets import FloatProgress

In [ ]:
average_error = fft_model.backtest(series_dfp_BoxCox,
                                   start=pd.Timestamp('20220226'),
                                   forecast_horizon=2,
                                   metric=smape,
                                   last_points_only=True,
                                   verbose=True)
median_error = fft_model.backtest(series_dfp_BoxCox,
                                  start=pd.Timestamp('20220226'),
                                  forecast_horizon=2,
                                  metric=smape,
                                  last_points_only=True,
                                  reduction=np.median,
                                  verbose=True)
print("Average error (MAPE) over all historical forecasts: {}".format(
    average_error))
print("Median error (MAPE) over all historical forecasts: {}".format(
    median_error))

In [ ]:
raw_errors = fft_model.backtest(series_dfp_BoxCox,
                                start=pd.Timestamp('20220226'),
                                forecast_horizon=2,
                                metric=smape,
                                last_points_only=True,
                                reduction=None,
                                verbose=True)

plt.hist(raw_errors)
plt.title("Individual error scores (histogram)")
plt.show()

In [ ]:
historical_fcast_fft = fft_model.historical_forecasts(
    series_dfp_BoxCox,
    start=pd.Timestamp('20220226'),
    forecast_horizon=2,
    last_points_only=True,
    verbose=True)

In [ ]:
historical_fcast_fft['0'].sum()

Let's see what this backtest forecast looks like. You can see it produces more accurate predictions at a 3 months horizon than the one-off prediction done above, because here the model is re-fit every month.

In [ ]:
series_dfp_BoxCox.plot(label='data')
historical_fcast_fft.plot(label='backtest 2-months ahead forecast (FFT)')
plt.title('MAPE = {:.2f}%'.format(mape(historical_fcast_fft,
                                       series_dfp_BoxCox)))
plt.legend()

Let's look at the fitted value residuals of our current `Theta` model, i.e. the difference between the 1-step forecasts at every point in time obtained by fitting the model on all previous points, and the actual observed values.

In [ ]:
plot_residuals_analysis(fft_model.residuals(series_dfp_BoxCox))

We can see that the distribution has a mean that is slightly larger than 0. This means that our `Theta` model is biased. We can also make out a large ACF value at lag equal to 12, which indicates that the residuals contain information that was not used by the model.



## Theta Model

In [ ]:

model = Theta()
parameters = {
    'theta': 2 - np.linspace(-10, 10, 50),
    'seasonality_period': [7, 14, 21],
    'season_mode': [SeasonalityMode.ADDITIVE],
}
model.gridsearch(parameters,
                 series_dfp_BoxCox,
                 metric=smape,
                 forecast_horizon=2)

In [ ]:
best_theta_model = Theta(theta=5.877551020408163,
                         seasonality_period=21,
                         season_mode=SeasonalityMode.ADDITIVE)
best_theta_model.fit(train_bx)
pred_best_theta = best_theta_model.predict(len(val_bx))

print('The MAPE is: {:.2f}.'.format(mape(val_bx, pred_best_theta)))

In [ ]:
train_bx.plot(label='train')
val_bx.plot(label='true')
pred_best_theta.plot(label='prediction', figsize=(15, 4))
plt.legend()

The residual analysis also reflects an improved performance in that we now have a distribution of the residuals centred at value 0, and the ACF values, although not insignificant, have lower magnitudes.

## Ensembling several predictions
*Ensembling* is about combining the forecasts produced by several models, in order to obtain a final -- and hopefully better forecast.

For instance, in our example of a "less naive" model above, we manually combined a naive seasonal model with a naive drift model. Here, we will try to find such combinations in an automated way, using `RegressionModel`s. A regression model is a model that predicts a *target* time series from a bunch of *features* time series. If the features time series are themselves obtained from forecasting models, their future (predicted) values can be combined using the regression model to obtain a final forecast.

Here, we will first compute the historical predictions two naive seasonal models (with 6 and 12 months seasonality), and naive drift model. To compute the historical forecasts, we can simply reuse the `historical_forecasts()` method:

In [ ]:
models = [
    FFT(nr_freqs_to_keep=7, trend='poly', trend_poly_degree=4),
    Theta(theta=5.877551020408163,
          seasonality_period=21,
          season_mode=SeasonalityMode.ADDITIVE),
    AutoARIMA()
]

model_predictions = [
    m.historical_forecasts(series_dfp_BoxCox,
                           start=pd.Timestamp('20201201'),
                           forecast_horizon=2,
                           last_points_only=True,
                           verbose=True) for m in models
]

In [ ]:

""" 
We build the regression model, and tell it to use the 12 preceding points to fit the regression
"""
regr_model = StandardRegressionModel(train_n_points=14)#, model = neigh )
""" 
Our target series is what we want to predict (the actual data)
    
    It has to have the same time index as the features series:
"""
series_target = series_dfp_BoxCox.slice_intersect(model_predictions[0])
""" 
Here we backtest our regression model
"""
ensemble_pred = regr_model.historical_forecasts(model_predictions,
                                                series_target,
                                                start=pd.Timestamp('20201203'),
                                                forecast_horizon=2,
                                                last_points_only=True,
                                                verbose=True)

Finally, let's see how good the regression performs, compared to the original forecasts:

In [ ]:
plt.figure(figsize=(16, 5))

series[100:].plot(label='actual')
# for i, m in enumerate(models):
#     BoxCox_dfp.inverse_transform(model_predictions[i]).plot(label=str(m))

#     # intersect last part, to compare all the methods over the duration of the ensemble forecast
#     model_pred = model_predictions[i].slice_intersect(ensemble_pred)

#     mape_model = mape(series, model_pred)
#     print('MAPE Error for {}: {:.2f}%'.format(m, mape_model))

print('MAPE Error ensemble: {:.2f}%'.format(mape(series, ensemble_pred)))

BoxCox_dfp.inverse_transform(ensemble_pred).plot(label='Ensemble')

print('\nCoefficients of the features time series:')
for i, m in enumerate(models):
    print('Learned coefficient for {}: {:.2f}'.format(
        m, regr_model.model.coef_[0][i]))
plt.legend()

In [ ]:
predictions = TimeSeries.pd_dataframe(BoxCox_dfp.inverse_transform(ensemble_pred))
predictions.columns = ['Label']

#### Error Analysis

In [ ]:
test = TimeSeries.pd_dataframe(series.split_after(pd.Timestamp('20201203'))[1][:-2])

In [ ]:
import seaborn as sns
error = (predictions['Label'] - test['Count']).astype('int').to_frame()
error.columns = ['errors']
errors_mean = error['errors'].mean()
errors_std = error['errors'].std()

fig, ax = plt.subplots(figsize=(7, 3))

sns.distplot(a=error['errors'], ax=ax, bins=15, rug=True)
ax.axvline(x=errors_mean, color='b', linestyle='--', label=r'$\mu$')
ax.axvline(x=errors_mean + 2 * errors_std,
           color='r',
           linestyle='--',
           label=r'$\mu \pm 2\sigma$')
ax.axvline(x=errors_mean - 2 * errors_std, color='k', linestyle='--')
ax.legend()
ax.set(title='Model Errors');

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

fig, ax = plt.subplots(2, 1, figsize=(8, 6))
plot_acf(x=error['errors'], ax=ax[0]),
plot_pacf(x=error['errors'], ax=ax[1]);